<a href="https://colab.research.google.com/github/thejsw/Quant/blob/portfolio/PythonPortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy-financial

In [2]:
!pip install pandas_datareader

In [3]:
!pip install yfinance

In [18]:
import numpy_financial as npf
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import yfinance as yf
import random
import math

# 1. 파이썬과 재무 기초 지식

## 1.2 현금흐름, 이자율과 시간 가치

이자율은 화폐를 보유하는 경우에 생기는 기회비용  
단리: 원금 x (1 + 이율 x 기간)  
복리: 원금 x (1 + 이율) ** 기간

In [5]:
a = 1
r = 1.0

n = 4
c_compound = a * (1+r / n) ** n

c_compound

2.44140625

## 1.3 NPV와 IRR

**NPV(순현재가치)**: 투자로부터 예상되는 미래 현금 흐름을 현재 가치로 환산한 후, 초기 투자 비용을 차감한 값  
-> NPV가 0보다 크면 투자할 가치가 있다고 판단함

In [6]:
# 현금흐름을 cashflows에 저장, i는 횟수, r은 이자율
cashflows = [12000, 15000, 18000, 21000, 26000]
i = 0
r = 0.015

# 최초 투자금액, 현금 유출이므로 (-)
npv = -70000

# cashflows 리스트를 반복해 미래가치를 현재가치로 계산해서 npv 변수에 누적
for c in cashflows:
  i = i+1
  npv = npv + c/(1+r) ** i

npv


17516.929512135568

In [7]:
# numpy_financial 라이브러리 사용
cashflows = [-70000, 12000, 15000, 18000, 21000, 26000]
r = 0.015

npv = npf.npv(r, cashflows)
npv

np.float64(17516.929512135568)

**IRR(내부수익률)**: 투자로부터 예상되는 현금 흐름의 현재 가치와 투자 비용이 같아지도록 만드는 할인율  
-> 투자를 통해 얻을 수 있는 수익률  
-> NPV를 0으로 만듦  
-> 예를 들어, IRR은 약 15.24%일 경우, 투자자가 요구하는 최소 수익률이 10%라면, 이 프로젝트는 투자할 가치가 있다고 판단

In [8]:
cashflows = [-70000, 12000, 15000, 18000, 21000, 26000]

irr = npf.irr(cashflows)
npv = npf.npv(irr, cashflows)

print('IRR {0:.1%} makes NPV {1:.0f}'.format(irr, npv))

IRR 8.7% makes NPV 0


## 1.4 수익률 대 수익률

**산술평균 (Arithmetic Mean)**  
일반적으로 "평균"이라고 부르는 값으로, 모든 값을 더한 후 값의 개수로 나눈 값  
  
-> 계산이 간단하고 직관적이나,   
복리 효과를 고려하지 않기 때문에 장기 투자 성과를 정확하게 반영X  
특히 수익률 변동성이 큰 경우, 실제 투자 수익률과 차이가 클 수 있음

**기하평균 (Geometric Mean)**  
값들을 모두 곱한 후 값의 개수에 해당하는 제곱근을 취한 값  
-> 투자 성과를 정확하게 측정하고, 여러 투자 안의 성과를 비교하는 데 유용

**지배원리 (dominance principle)**  
포트폴리오에서 얻는 기대수익률은 클수록 좋고, 표준편차와 분산으로 표현하는 위험은 작을수록 좋음

## 1.5 자주 사용하는 통계량

**이동평균**  
계산에 들어가는 값 중 가장 오래된 값을 버리고, 새로운 값을 추가하여 구한 평균

In [9]:
prices = [44800, 44850, 44600, 43750, 44000, 43900, 44350, 45350, 45500, 45700]

# 5일 이동평균
n = 5

for p in prices[n: ]:
  end_index = prices.index(p)
  begin_index = end_index - n
  print(sum(prices[begin_index:end_index])/n)

44400.0
44220.0
44120.0
44270.0
44620.0


**가중평균**  
자료에 가중치를 고려해 구한 평균  
-> 예를 들어, A종목 100주를 주당 5,000원에 매입하고, 다시 50주를 7,500원에 추가 매수했다면,  
5,000원 매입단가의 비중과 7,500원 매입단가의 비중을 반영해 가중평균으로 계산

In [10]:
# sizes는 매입수량, prices는 매입단가
sizes = [100, 50, 80]
prices = [5,000, 7,500, 6,000]

# wgt_avg는 합계를 저장할 변수
wgt_avg = 0.0

# sizes와 prices를 zip함수로 묶어 for 루프로 반복함
for s, p in zip(sizes, prices):
  wgt_avg += s*p

print(wgt_avg)

1060.0


**공분산**   
두 변수 간의 선형적인 관계  
Cov(X, Y) = Σ [(Xi - X̄) * (Yi - Ȳ)] / (n - 1)  

**상관관계**  
공분산의 한계를 극복하기 위해 상관관계라는 개념을 사용  
공분산을 각 변수의 표준편차로 나누어 -1과 1 사이의 값으로 정규화

In [11]:
# 평균을 계산하는 함수
def mean(x):
  return sum(x) / len(x)

# 두 리스트 곱의 합계
def sum_of_product(xs, ys):
  return sum(x*y for x, y in zip(xs, ys))

# 제곱합을 계산하는 함수
def sum_of_squares(y):
  return sum_of_squares(v, v)

# 편차를 계산하는 함수
def deviation(xs):
  x_mean = mean(xs)
  return [x - x_mean for x in xs]

# 분산을 계산하는 함수
def variance(x):
  n = len(x)
  deviations = deviation(x)
  return sum_of_squares(deviations) / (n-1)

# 공분산을 계산하는 함수
def convariance(x, y):
  n = len(x)
  deviations = deviation(x)
  return sum_of_squares(deviations) / (n-1)

# 표준편차를 계산하는 함수
def standard_deviation(x):
  return math.sqrt(variance(x))

# 상관계수를 계산하는 함수
def correlation(xs, ys):
  stdev_x = standard_deviation(xs)
  stdev_y = standard_deviation(ys)
  if stdev_x > 0 and stdev_y > 0:
    return convariance(xs, ys) / (stdev_x, stdev_y)
  else:
    return 0

# 2. 투자와 자산배분

> 자산은 리스크가 없는 자산과 리스크가 있는 자산으로 나뉜다.   
> 리스크가 없는 자산으로만 구성된 포트폴리오는 투자자가 원하는 기대수익률을 만족시켜주지 못한다.  
> 따라서 자산을 위험자산과 무위험자산으로 분류하고 *좋은 위험자산 포트폴리오를 찾아내는*, **효율적 포트폴리오**를 구성해야 한다.



## 2.3 포트폴리오 성과 측정 삼총사

**1. 샤프지수**  
- 개념: 샤프 지수는 총 위험(변동성) 대비 초과 수익률을 나타내는 지표  
즉, 투자자가 감수한 위험 대비 얼마나 많은 초과 수익을 얻었는지 측정  

`샤프 지수 = (포트폴리오 수익률 - 무위험 수익률) / 포트폴리오 표준편차(총 위험)`  

- 해석: 높은 샤프 지수는 위험 대비 수익률이 높다는 것을 의미함. 일반적으로 샤프 지수가 1 이상이면 좋은 투자로 간주  

**2. 젠센알파지수**  
- 개념: 포트폴리오의 실제 수익률과 기대수익률의 차이  
즉, 시장 대비 얼마나 높은 성과를 냈는지 측정

`젠센 알파 = 포트폴리오 수익률 - 기대(적정)수익률`  

- 해석: 양수이면 포트폴리오가 CAPM(자본자산가격결정모델)에서 예측하는 것보다 높은 수익률을 달성

**3. 트레이너지수**  
- 개념: 체계적 위험(베타) 대비 초과 수익률을 나타내는 지표
즉, 투자자가 감수한 체계적 위험 단위당 얼마나 많은 초과 수익을 얻었는지 측정

`트레이너 지수 = (포트폴리오 수익률 - 무위험 수익률) / β`  
베타: 시장 수익률 변동에 대한 포트폴리오 수익률의 민감도(체계적 위험)  

- 해석: 높은 트레이너 지수는 체계적 위험 대비 수익률이 높다는 것을 의미

**분산 투자가 잘 되어 있지 않은 포트폴리오**: 샤프 지수가 더 적합  
**분산 투자가 잘 되어 있고, 체계적 위험이 중요한 포트폴리오**: 트레이너 지수가 더 적합  
**특정 벤치마크와 비교하여 초과 수익을 평가하고 싶을 때**: 젠센 알파 지수가 더 적합  

**정보비율**  
투자 성과를 평가하는 지표  
정보비율이 높을수록 더 많은 초과 수익을 얻음

`정보비율 (IR) = 초과 수익률(포트폴리오 수익률 - 벤치마크 수익률) / 추적 오차(표준편차)`





**최대 낙폭(MDD)**  
고점에서 저점까지 하락폭이 가장 큰 구간의 등락률  
즉, 투자자의 최대손실가능 수익률을 의미

`MDD = 기간 중 최저 가치 - 최고 가치 / 최고 가치`

In [12]:
values = [100, 120, 130, 100, 65, 80, 100, 120, 140, 160]

def mdd(x):
  arr = np.array(x)
  idx_lower = np.argmin(arr - np.maximum.accumulate(arr))
  idx_upper = np.argmax(arr[:idx_lower])
  return (arr[idx_lower] - arr[idx_upper]) / arr[idx_upper]

print("{:.2%}".format(mdd(values)))

-50.00%


# 3. 평균-분산 포트폴리오 이론

## 3.1 포트폴리오의 기대수익과 위험

> 노벨 경제학상 수상자 해리 마코위츠는 투자자들이 주가 대폭락 이후 수익뿐만 아니라 리스크에도 관심을 가지는 것을 목격했다.  
그는 **상관계수가 낮은 자산을 결합**해 위험을 줄이며 수익률을 높일 수 있는 최적 포트폴리오를 제시했다.


### 3.1.1 두 개 주식으로 구성된 포트폴리오
확률분포 없이 조건을 단순화해 포트폴리오의 기대수익률을 구해볼 수 있다

**1. 포트폴리오 기대수익률** = A 투자 비중 * A 기대수익률 + B 투자 비중 * B 기대수익률

In [13]:
# 각각 호황시, 평상시, 불황시 수익률과 확률
stock_a = [0.07, 0.04, 0.02]
stock_b = [0.15, 0.07, -0.07]
prob = [1/3, 1/3, 1/3]

# 주식 a,b의 경기 국면에 따른 수익률 기댓값을 저장할 변수
ex_a = 0.0
ex_b = 0.0
wgt_a = 0.5
wgt_b = 0.5

# 주식 a,b의 기대수익률
ex_a = sum(s*p for s,p in zip(stock_a, prob))
ex_b = sum(s*p for s,p in zip(stock_b, prob))

# 포트폴리오 기대수익률
ex_portfolio = wgt_a * ex_a + wgt_b * ex_b

print('주식 a의 기대수익률은 {:.2%}'.format(ex_a))
print('주식 b의 기대수익률은 {:.2%}'.format(ex_b))
print('포트폴리오 기대수익률은 {:.2%}'.format(ex_portfolio))

주식 a의 기대수익률은 4.33%
주식 b의 기대수익률은 5.00%
포트폴리오 기대수익률은 4.67%


**2. 행렬 연산**으로 구해보는 포트폴리오 기대수익률

In [14]:
# matrix()를 통해 확률, 주식 a,b의 수익률을 1x3 행렬로 만듦
stock_a = np.array([[0.07, 0.04, 0.02]])
stock_b = np.array([[0.15, 0.07, -0.07]])
prob = np.array([[1/3, 1/3, 1/3]])

# 행렬 곱하기 연산 수행, 행렬 차원을 맞추기 위해 전치 사용
# prob는 1x3, stock_a, stock_b는 3x1이 되어 1x1의 행렬이 나오게 됨
ex_a = prob @ stock_a.T
ex_b = prob @ stock_b.T
ex_portfolio = wgt_a * ex_a + wgt_b * ex_b

print('주식 a의 기대수익률은 {:.2%}'.format(ex_a[0, 0]))
print('주식 b의 기대수익률은 {:.2%}'.format(ex_b[0, 0]))
print('포트폴리오 기대수익률은 {:.2%}'.format(ex_portfolio[0, 0]))

주식 a의 기대수익률은 4.33%
주식 b의 기대수익률은 5.00%
포트폴리오 기대수익률은 4.67%


**3. 포트폴리오 위험** = 주식A 투자 비중** x 주식A 분산 + 주식B 투자비중** + 주식B 분산 + 2 x 주식A 투자 비중 x 주식B 투자 비중 x 포트폴리오 공분산

In [15]:
# 1. 변수 선언 및 주식 a,b의 분산 구하기

# 각각 호황시, 평상시, 불황시 수익률과 확률
stock_a = [0.07, 0.04, 0.02]
stock_b = [0.15, 0.07, -0.07]
prob = [1/3, 1/3, 1/3]

# 주식 a,b의 경기 국면에 따른 수익률 기댓값을 저장할 변수
ex_a = 0.0
ex_b = 0.0
wgt_a = 0.5
wgt_b = 0.5

# 분산을 저장할 변수 준비
var_a, var_b = 0.0, 0.0

# 확률과 편차 제곱을 곱해 각각 분산으로 저장
var_a = sum(p*(s-ex_a)**2 for p,s in zip(prob, stock_a))
var_b = sum(p*(s-ex_b)**2 for p,s in zip(prob, stock_b))

print('주식 a의 분산은 {:.2%}'.format(var_a))
print('주식 b의 분산은 {:.2%}'.format(var_b))

주식 a의 분산은 0.23%
주식 b의 분산은 1.08%


In [16]:
# 2. 포트폴리오 위험(분산) 측정을 위한 공분산, 표준편차 계산

cov = sum(p*(a-ex_a)*(b-ex_b) for p,a,b in zip(prob, stock_a, stock_b))
var_p = wgt_a**2 * var_a + wgt_b**2 * var_b + 2*wgt_a*wgt_b*cov
std_p = math.sqrt(var_p)

print('포트폴리오의 위험(분산)은 {:.2%}'.format(var_p))
print('포트폴리오의 표준편차는 {:.2%}'.format(std_p))

포트폴리오의 위험(분산)은 0.53%
포트폴리오의 표준편차는 7.25%


### 3.1.2 Pandas를 이용해 실제 데이터로 포트폴리오 기대수익률 계산

In [17]:
tickers = ['MMM', 'ADBE', 'AMD', 'GOOGL', 'GOOG', 'AMZN']

adjClose = pd.DataFrame()

adjClose = yf.download(tickers, start='2023-01-01')

dailySimpleReturns = adjClose.pct_change()
meanReturns = np.matrix(dailySimpleReturns.mean()).T

print(meanReturns)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  6 of 6 completed

[[0.00040233]
 [0.00120981]
 [0.00155814]
 [0.00121624]
 [0.00120916]
 [0.00087676]
 [0.00032367]
 [0.00105789]
 [0.00151413]
 [0.0011772 ]
 [0.0011709 ]
 [0.00084622]
 [0.0003861 ]
 [0.00107691]
 [0.00154657]
 [0.0011886 ]
 [0.00118122]
 [0.00083435]
 [0.00038719]
 [0.00112578]
 [0.0016179 ]
 [0.00123832]
 [0.00122689]
 [0.00085314]
 [0.06513982]
 [0.04642759]
 [0.04641777]
 [0.05624512]
 [0.05566886]
 [0.11038627]]


## 3.3 체계적 위험과 비체계적 위험  

- 비체계적 위험: 분산 투자로 제거할 수 있는 위험  
개별 주식과 관련된 고유의 위험으로, 어닝쇼크(실적부진), 소송, 노사분규와 같은 것
- 체계적 위험: 분산 투자로 제거할 수 없는 위험  
시장 전체와 관련된 위험으로 경제변수(이자율, 환율, 경기선행지수, 실업률, 경제정책 등)의 불리한 움직임

## 3.4 무위험자산과 최적 자산배분  

- 무위험자산: 이자율, 인플레이션의 변화에도 영향을 받지 않아, 미래의 현금흐름에 불확실성이 없는 자산을 의미함  
예를 들어, 만기가 짧아 인플레이션, 이자율의 영향을 적게 받는 정부발행채권,  
또는 인플레이션의 영향을 받지만 만기가 긴 10년 국공채가 있음

## 3.5 최적 포트폴리오 선택  
- 투자자의 위험회피 성향을 나타내는 **무차별곡선**과 투자기회집합의 **효율적투자선**이 접하는 최적 포트폴리오를 나타냄

In [22]:
tickers = ['AAPL', 'F', 'AMZN', 'GE', 'TSLA']

adjClose = pd.DataFrame()

adjClose = yf.download(tickers, start='2023-01-01')
adjClose.head(5)

[*********************100%***********************]  5 of 5 completed


Price            Close                                               \
Ticker            AAPL       AMZN          F         GE        TSLA   
Date                                                                  
2023-01-03  123.632523  85.820000   9.566105  52.341320  108.099998   
2023-01-04  124.907707  85.139999   9.836378  55.387806  113.639999   
2023-01-05  123.583099  83.120003  10.032943  56.247822  110.339996   
2023-01-06  128.130234  86.080002  10.303217  56.760670  113.059998   
2023-01-09  128.654129  87.360001  10.393308  57.336632  119.769997   

Price             High                                               ...  \
Ticker            AAPL       AMZN          F         GE        TSLA  ...   
Date                                                                 ...   
2023-01-03  129.395510  86.959999   9.762668  52.409071  118.800003  ...   
2023-01-04  127.181276  86.980003   9.893709  55.387806  114.589996  ...   
2023-01-05  126.301493  85.419998  10.139415  56.452964  111.750000  ...   
2023-01-06  128.792531  86.400002  10.311407  57.068379  114.389999  ...   
2023-01-09  131.876670  89.480003  10.589872  58.314991  123.519997  ...   

Price             Open                                               \
Ticker            AAPL       AMZN          F         GE        TSLA   
Date                                                                  
2023-01-03  128.782641  85.459999   9.680767  51.768509  118.470001   
2023-01-04  125.431615  86.550003   9.729906  53.975494  109.110001   
2023-01-05  125.668849  85.330002   9.918280  55.151112  110.510002   
2023-01-06  124.561732  83.029999   9.926470  56.815900  103.000000   
2023-01-09  128.970458  87.459999  10.434259  56.808001  118.959999   

Price          Volume                                           
Ticker           AAPL      AMZN         F        GE       TSLA  
Date                                                            
2023-01-03  112117500  76706000  45809000  10280781  231402800  
2023-01-04   89113600  68885100  53429700  21031104  180389000  
2023-01-05   80962700  67930800  50785600  16001061  157986300  
2023-01-06   87754700  83303400  53089100  13018044  220911100  
2023-01-09   70790800  65266100  50865500   9488844  190284000  

[5 rows x 25 columns]

In [25]:
ret_daily = adjClose.pct_change()
ret_daily.head(5)

Price          Close                                              High  \
Ticker          AAPL      AMZN         F        GE      TSLA      AAPL   
Date                                                                     
2023-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2023-01-04  0.010314 -0.007924  0.028253  0.058204  0.051249 -0.017112   
2023-01-05 -0.010605 -0.023726  0.019983  0.015527 -0.029039 -0.006918   
2023-01-06  0.036794  0.035611  0.026939  0.009118  0.024651  0.019723   
2023-01-09  0.004089  0.014870  0.008744  0.010147  0.059349  0.023947   

Price                                               ...      Open            \
Ticker          AMZN         F        GE      TSLA  ...      AAPL      AMZN   
Date                                                ...                       
2023-01-03       NaN       NaN       NaN       NaN  ...       NaN       NaN   
2023-01-04  0.000230  0.013423  0.056836 -0.035438  ... -0.026021  0.012755   
2023-01-05 -0.017935  0.024835  0.019231 -0.024784  ...  0.001891 -0.014096   
2023-01-06  0.011473  0.016963  0.010901  0.023624  ... -0.008810 -0.026954   
2023-01-09  0.035648  0.027006  0.021844  0.079815  ...  0.035394  0.053354   

Price                                       Volume                      \
Ticker             F        GE      TSLA      AAPL      AMZN         F   
Date                                                                     
2023-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2023-01-04  0.005076  0.042632 -0.079007 -0.205177 -0.101959  0.166358   
2023-01-05  0.019360  0.021781  0.012831 -0.091466 -0.013854 -0.049487   
2023-01-06  0.000826  0.030186 -0.067958  0.083890  0.226298  0.045357   
2023-01-09  0.051155 -0.000139  0.154951 -0.193310 -0.216525 -0.041884   

Price                           
Ticker            GE      TSLA  
Date                            
2023-01-03       NaN       NaN  
2023-01-04  1.045672 -0.220455  
2023-01-05 -0.239172 -0.124191  
2023-01-06 -0.186426  0.398293  
2023-01-09 -0.271101 -0.138640  

[5 rows x 25 columns]

In [26]:
ret_annual = ret_daily.mean() * 250
ret_annual

Price   Ticker
Close   AAPL       0.262983
        AMZN       0.389534
        F          0.086601
        GE         0.624535
        TSLA       0.610266
High    AAPL       0.239101
        AMZN       0.378531
        F          0.071284
        GE         0.616809
        TSLA       0.540745
Low     AAPL       0.258019
        AMZN       0.386643
        F          0.080244
        GE         0.618792
        TSLA       0.569691
Open    AAPL       0.251182
        AMZN       0.404475
        F          0.083717
        GE         0.625831
        TSLA       0.598050
Volume  AAPL      12.283442
        AMZN      11.604441
        F         13.827640
        GE        16.414591
        TSLA       6.563683
dtype: float64

In [27]:
cov_daily = ret_daily.cov()
cov_annual = cov_daily * 250
cov_annual

Price             Close                                              High  \
Ticker             AAPL      AMZN         F        GE      TSLA      AAPL   
Price  Ticker                                                               
Close  AAPL    0.069139  0.041452  0.026523  0.027637  0.076324  0.046181   
       AMZN    0.041452  0.104438  0.029265  0.040039  0.088089  0.020505   
       F       0.026523  0.029265  0.130729  0.028251  0.069933  0.017961   
       GE      0.027637  0.040039  0.028251  0.088618  0.059765  0.014441   
       TSLA    0.076324  0.088089  0.069933  0.059765  0.388264  0.041567   
High   AAPL    0.046181  0.020505  0.017961  0.014441  0.041567  0.054580   
       AMZN    0.020354  0.069829  0.017514  0.024169  0.051970  0.022331   
       F       0.010274  0.010522  0.087786  0.014504  0.041745  0.017872   
       GE      0.014961  0.021817  0.019869  0.057861  0.030870  0.015658   
       TSLA    0.043003  0.042470  0.046066  0.029715  0.259215  0.048238   
Low    AAPL    0.045397  0.019362  0.015034  0.016367  0.050835  0.043766   
       AMZN    0.022024  0.070811  0.017667  0.028927  0.063219  0.018583   
       F       0.008725  0.013682  0.091377  0.015955  0.038974  0.013357   
       GE      0.012452  0.021703  0.017275  0.054646  0.038176  0.014176   
       TSLA    0.038686  0.040964  0.039519  0.031933  0.258995  0.036874   
Open   AAPL    0.020075 -0.000369  0.010563  0.008026  0.015495  0.042034   
       AMZN    0.004001  0.033707  0.007019  0.015739  0.029704  0.018672   
       F      -0.003695  0.000910  0.045047  0.005548  0.023334  0.011605   
       GE      0.002561  0.006429  0.008846  0.019189  0.017188  0.013980   
       TSLA    0.005656  0.004778  0.018867  0.008933  0.140199  0.035751   
Volume AAPL   -0.047546 -0.005906  0.013681 -0.107495 -0.273965  0.192914   
       AMZN   -0.027100  0.064419 -0.029609 -0.103147 -0.041283  0.070425   
       F      -0.039369 -0.076076 -0.342465 -0.111449 -0.034484  0.029126   
       GE     -0.062422 -0.090350 -0.066238  0.189084 -0.401145 -0.025911   
       TSLA    0.018496  0.035691  0.076614 -0.016689  0.324341  0.084076   

Price                                                  ...      Open  \
Ticker             AMZN         F        GE      TSLA  ...      AAPL   
Price  Ticker                                          ...             
Close  AAPL    0.020354  0.010274  0.014961  0.043003  ...  0.020075   
       AMZN    0.069829  0.010522  0.021817  0.042470  ... -0.000369   
       F       0.017514  0.087786  0.019869  0.046066  ...  0.010563   
       GE      0.024169  0.014504  0.057861  0.029715  ...  0.008026   
       TSLA    0.051970  0.041745  0.030870  0.259215  ...  0.015495   
High   AAPL    0.022331  0.017872  0.015658  0.048238  ...  0.042034   
       AMZN    0.082075  0.018280  0.025708  0.054922  ...  0.015826   
       F       0.018280  0.105553  0.018478  0.056551  ...  0.021039   
       GE      0.025708  0.018478  0.072396  0.034826  ...  0.015288   
       TSLA    0.054922  0.056551  0.034826  0.303213  ...  0.044598   
Low    AAPL    0.018384  0.014658  0.015021  0.048767  ...  0.053781   
       AMZN    0.070053  0.017880  0.027299  0.053480  ...  0.024508   
       F       0.018167  0.083906  0.019804  0.049062  ...  0.024732   
       GE      0.025215  0.019490  0.049519  0.033811  ...  0.027273   
       TSLA    0.046868  0.049797  0.032336  0.247559  ...  0.053017   
Open   AAPL    0.015826  0.021039  0.015288  0.044598  ...  0.081064   
       AMZN    0.067564  0.021729  0.028749  0.059238  ...  0.044848   
       F       0.016078  0.084727  0.017382  0.055740  ...  0.034813   
       GE      0.022186  0.020554  0.048240  0.037116  ...  0.033567   
       TSLA    0.045079  0.053792  0.033737  0.255916  ...  0.082982   
Volume AAPL    0.026899  0.029579 -0.055150 -0.082660  ... -0.080319   
       AMZN    0.149231  0.027680 -0.042943 -0.032838  ... -0.070784   
       F      -0.036913  0.124155 -0.083424

In [28]:
p_returns = []
p_volatility = []
p_weights = []
n_assets = len(tickers)
n_ports = 30000

for s in range(n_ports):
  wgt = np.random.random(n_assets)
  wgt /= np.sum(wgt)
  ret = np.dot(wgt, ret_annual)
  vol = np.sqrt(np.dot(wgt.T, np.dot(cov_annual, wgt)))

  p_returns.append(ret)
  p_volatility.append(vol)
  p_weights.append(wgt)

p_returns = np.array(p_returns)
p_volatility = np.array(p_volatility)

ValueError: shapes (5,) and (25,) not aligned: 5 (dim 0) != 25 (dim 0)